In [68]:
from PIL import Image
import numpy as np

def preprocess(pil_img, scale):
    w, h = pil_img.size
    newW, newH = int(scale * w), int(scale * h)
    assert newW > 0 and newH > 0, 'Scale is too small'
    pil_img = pil_img.resize((newW, newH))

    img_nd = np.array(pil_img)

    if len(img_nd.shape) == 2:
        img_nd = np.expand_dims(img_nd, axis=2)

    # HWC to CHW
    img_trans = img_nd.transpose((2, 0, 1))
    if img_trans.max() > 1:
        img_trans = img_trans / 255

    return img_trans

In [69]:
im = preprocess(gt, .5)

In [103]:
import shutil

for s, f in zip(areas, lf):
    if s > 0:
        f2 = f.split('.')[0] + '.jpg'
        shutil.copy(dr + f, 'polyp_data/train2/mask/' + f)
        shutil.copy('polyp_data/train/img/' + f2, 'polyp_data/train2/img/' + f2)

In [137]:
import os
dr = 'polyp_data/train2/mask/'
lf = os.listdir(dr)
areas = []
for i in lf:
    with Image.open(dr + i) as im:
        arr = np.array(im)
        areas.append(arr[arr==255].shape[0] / arr.reshape((-1)).shape[0])

In [138]:
np.array(areas).mean()

0.011355858983583506

In [114]:
from unet_pytorch.dice_loss import dice_coeff

In [118]:
import torch
dr = 'polyp_data/test/mask/'
lf = os.listdir(dr)
def toarr(im):
    im = np.array(im).astype(np.float) / 255.
    return torch.tensor(im) 
coeff = []
for i in lf:
    pred = i.split('.')[0] + '_OUT.png'
    im_pred = Image.open(dr + '../pred/' + pred)
    im_gt = Image.open(dr + i)
    coeff.append(dice_coeff(toarr(im_gt), toarr(im_pred)).item())
    im_pred.close()
    im_gt.close()
coeff = np.array(coeff)

In [124]:
coeff.mean()

0.8762475280891168

In [128]:
(coeff > .9).sum() / coeff.shape[0]

0.38087520259319285

In [125]:
(coeff > .8).sum() / coeff.shape[0]

0.8646677471636953

In [126]:
(coeff > .5).sum() / coeff.shape[0]

0.9983792544570502

In [136]:
coeff.

array([0.88252813, 0.86617124, 0.82870382, ..., 0.97037047, 0.87393039,
       0.77085143])